# K-Means Clustering Notebook

## About

With this project I am to use the K-Means clustering algorithm on the country dataset. The dataset attributes include columns such as child mortality, net income per person  life expectancy and other factors that determine the overall status of a country. After running the dataset through a model, we will determine which countries would make better candidates for extra aid. 

## Setup

### Import libraries and dataset

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
data = pd.read_csv('../datasets/mall_customer_segmentation_data/Mall_Customers.csv')

In [3]:
data.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [4]:
data.shape

(200, 5)

### Data Exploration

In [5]:
data.columns

Index(['CustomerID', 'Gender', 'Age', 'Annual Income (k$)',
       'Spending Score (1-100)'],
      dtype='object')

In [6]:
data.isnull().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

### Graph Data